In [1]:
import sys
print("현재 Python 실행 경로:", sys.executable)

현재 Python 실행 경로: C:\Users\SSAFY\AppData\Local\Programs\Python\Python311\python.exe


In [2]:
# SPARK_HOME을 311로 변경

import os 

os.environ["SPARK_HOME"] = r"C:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark"

# 변경된 값 확인
print("SPARK_HOME:", os.environ.get("SPARK_HOME"))

SPARK_HOME: C:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark


In [3]:
import os
os.environ["PYSPARK_PYTHON"] = r"C:\Users\SSAFY\AppData\Local\Programs\Python\Python311\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"] = os.environ["PYSPARK_PYTHON"]

# 설정 확인
print("PYSPARK_PYTHON:", os.environ.get("PYSPARK_PYTHON"))
print("PYSPARK_DRIVER_PYTHON:", os.environ.get("PYSPARK_DRIVER_PYTHON"))

print("JAVA_HOME:", os.environ.get("JAVA_HOME"))
print("SPARK_HOME:", os.environ.get("SPARK_HOME"))
print("HADOOP_HOME:", os.environ.get("HADOOP_HOME"))

PYSPARK_PYTHON: C:\Users\SSAFY\AppData\Local\Programs\Python\Python311\python.exe
PYSPARK_DRIVER_PYTHON: C:\Users\SSAFY\AppData\Local\Programs\Python\Python311\python.exe
JAVA_HOME: C:\Program Files\Eclipse Adoptium\jdk-17.0.14.7-hotspot
SPARK_HOME: C:\Users\SSAFY\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark
HADOOP_HOME: None


In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("SparkSQL").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(spark)

In [6]:
region_data = [
    ("경기도", "기장군"), ("경기도", "달성군"), ("경기도", "강화군"), ("경기도", "옹진군"), ("경기도", "울주군"), 
    ("경기도", "세종시"), ("경기도", "평택시"), ("경기도", "남양주시"), ("경기도", "용인시"), ("경기도", "파주시"),
    ("경기도", "이천시"), ("경기도", "안성시"), ("경기도", "김포시"), ("경기도", "화성시"), ("경기도", "광주시"),
    ("경기도", "양주시"), ("경기도", "포천시"), ("경기도", "여주시"), ("경기도", "연천군"), ("경기도", "가평군"), 
    ("경기도", "양평군")
]

# PySpark DataFrame 생성
df_regions = spark.createDataFrame(region_data, ["대분류", "소분류"])

# 데이터 확인
df_regions.show()

+------+--------+
|대분류|  소분류|
+------+--------+
|경기도|  기장군|
|경기도|  달성군|
|경기도|  강화군|
|경기도|  옹진군|
|경기도|  울주군|
|경기도|  세종시|
|경기도|  평택시|
|경기도|남양주시|
|경기도|  용인시|
|경기도|  파주시|
|경기도|  이천시|
|경기도|  안성시|
|경기도|  김포시|
|경기도|  화성시|
|경기도|  광주시|
|경기도|  양주시|
|경기도|  포천시|
|경기도|  여주시|
|경기도|  연천군|
|경기도|  가평군|
+------+--------+
only showing top 20 rows



In [8]:
from pyspark.sql import SparkSession

# Spark 세션 시작
spark = SparkSession.builder.appName("FARM Recommendation").getOrCreate()

# 절대 경로로 수정
file_path = "FARM_normalized.csv"

try:
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    print("파일 로딩 성공!")
    df.show(5)
except Exception as e:
    print(f"파일 로딩 실패: {e}")

파일 로딩 성공!
+------+------+-------------+------------------+------------------+-------------------+-------------------+-------------------+------------------+--------------------+------------------+------------------+------------------+
|대분류|소분류|  full_region|          공시지가| ㎡당 거래가(만원)|     소비자물가지수|     면적당 학교 수|     면적당 병원 수|  인구밀도 (명/㎢)|커뮤니티 활성화 지수|       대기질 지수|         수질 지수|         녹지 비율|
+------+------+-------------+------------------+------------------+-------------------+-------------------+-------------------+------------------+--------------------+------------------+------------------+------------------+
|경기도|기장군|경기도 기장군|0.4450993595175989|1.4203759292283242|0.23636363636363455| 1.1673161859158605| 1.8633794568358808| 1.798780368001232|  0.0403409767651533|0.3633770368536448|0.5916159466316668|0.3149321009320848|
|경기도|달성군|경기도 달성군|0.4450993595175989|1.4203759292283242|0.23636363636363455| 1.0109128467818698| 1.3982114068424314|1.4493267276786141| 0.04887181804916576|0.388222215

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, array
from pyspark.sql.types import FloatType
import numpy as np

# Spark 세션 시작
spark = SparkSession.builder.appName("FARM Recommendation").getOrCreate()

# CSV 파일 불러오기
file_path = "FARM_normalized_updated.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# 사용자 선호도 입력
user_preferences = {
    "F": 0.2,  # 비용 선호도 (0 ~ 1)
    "A": 0.2,  # 접근성 선호도 (0 ~ 1)
    "R": 0.3,  # 커뮤니티 활성화 선호도 (0 ~ 1)
    "M": 0.8   # 자연환경 선호도 (0 ~ 1)
}

# 사용자 벡터를 10차원으로 확장
user_vector = np.array([
    user_preferences["F"],  # 공시지가
    user_preferences["F"],  # ㎡당 거래가(만원)
    user_preferences["F"],  # 소비자물가지수
    user_preferences["A"],  # 면적당 학교 수
    user_preferences["A"],  # 면적당 병원 수
    user_preferences["R"],  # 인구밀도 (명/㎢)
    user_preferences["R"],  # 커뮤니티 활성화 지수
    user_preferences["M"],  # 대기질 지수
    user_preferences["M"],  # 수질 지수
    user_preferences["M"]   # 녹지 비율
])

# 유사도 계산 함수 (코사인 유사도)
def cosine_similarity(arr1, arr2):
    dot_product = np.dot(arr1, arr2)
    norm1 = np.linalg.norm(arr1)
    norm2 = np.linalg.norm(arr2)
    return float(dot_product / (norm1 * norm2)) if norm1 != 0 and norm2 != 0 else 0.0

# UDF로 유사도 계산 함수 등록
@udf(FloatType())
def calculate_similarity(*features):
    data_vector = np.array(features, dtype=float)
    return cosine_similarity(data_vector, user_vector)

# 벡터화
features = ["공시지가", "㎡당 거래가(만원)", "소비자물가지수", "면적당 학교 수", "면적당 병원 수", "인구밀도 (명/㎢)", "커뮤니티 활성화 지수", "대기질 지수", "수질 지수", "녹지 비율"]
df = df.withColumn("similarity", calculate_similarity(*[col(f) for f in features]))


# 유사도 기반 추천 (상위 3개 지역)
recommended = df.select("full_region", "similarity").orderBy(col("similarity").desc()).limit(3)
recommended.show()


+---------------+----------+
|    full_region|similarity|
+---------------+----------+
|경상남도 남해군|0.96840334|
|충청북도 음성군|0.95567137|
|제주도 서귀포시| 0.9549242|
+---------------+----------+



In [8]:
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType
import numpy as np


# Spark 세션 시작
spark = SparkSession.builder.appName("FARM Recommendation").getOrCreate()

start_time = time.time()  # 시간 측정 시작

# CSV 파일 불러오기
file_path = "FARM_normalized_updated.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# 사용자 선호도 입력
user_preferences = {
    "F": 0.2,
    "A": 0.2,
    "R": 0.3,
    "M": 0.8
}

user_vector = np.array([
    user_preferences["F"],
    user_preferences["F"],
    user_preferences["F"],
    user_preferences["A"],
    user_preferences["A"],
    user_preferences["R"],
    user_preferences["R"],
    user_preferences["M"],
    user_preferences["M"],
    user_preferences["M"]
])

def cosine_similarity(arr1, arr2):
    dot_product = np.dot(arr1, arr2)
    norm1 = np.linalg.norm(arr1)
    norm2 = np.linalg.norm(arr2)
    return float(dot_product / (norm1 * norm2)) if norm1 != 0 and norm2 != 0 else 0.0

@udf(FloatType())
def calculate_similarity(*features):
    data_vector = np.array(features, dtype=float)
    return cosine_similarity(data_vector, user_vector)

features = ["공시지가", "㎡당 거래가(만원)", "소비자물가지수", "면적당 학교 수", "면적당 병원 수", 
            "인구밀도 (명/㎢)", "커뮤니티 활성화 지수", "대기질 지수", "수질 지수", "녹지 비율"]
df = df.withColumn("similarity", calculate_similarity(*[col(f) for f in features]))

recommended = df.select("full_region", "similarity").orderBy(col("similarity").desc()).limit(3)
recommended.show()

end_time = time.time()  # 시간 측정 종료

print(f"실행 시간: {end_time - start_time:.2f}초")


+---------------+----------+
|    full_region|similarity|
+---------------+----------+
|경상남도 남해군|0.96840334|
|충청북도 음성군|0.95567137|
|제주도 서귀포시| 0.9549242|
+---------------+----------+

실행 시간: 1.07초
